# Person re-id 
Before using this notebook it is expected that an HDF5 file is created. See notebook "create hdf5.ipynb"

## Set-up
first, import packages

In [0]:
# import packages
import h5py
import os
import pandas as pd
from google.colab import drive
import cv2
import datetime as dt
from PIL import Image
import numpy as np
import sys
import logging
import time
import itertools
import torch
import torch.nn as nn
import torchvision
from torch.utils.data import DataLoader
import pickle
from tqdm import tqdm
from logger import logger
from torchsummary import summary

In [0]:
#mount the drive to be able to access images, functions and classes
drive.mount('/content/drive', force_remount=True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
os.chdir("/content/drive/My Drive/Thesis re-id/triplet-reid-master")

In [0]:
# Loading local classes and functions
from own_code.backbone_normal import EmbedNetwork
from own_code.loss import TripletLoss
from own_code.triplet_selector import BatchHardTripletSelector
from own_code.batch_sampler import BatchSampler
from own_code.Market1501 import Market1501
from own_code.optimizer import AdamOptimWrapper
#import the anti-aliased networks
from models_lpf import *
import models_lpf.resnet

## Loading and preprocessing data
The labels are stored in an csv file. These labels have to be loaded. 

In [0]:
# Labels
labels = pd.read_csv('data/market1501_train.csv', names = ['pid', 'fid'], header = None, dtype = str)
labels_query = pd.read_csv('data/market1501_query.csv', names = ['pid', 'fid'], header = None, dtype = str)
labels_test = pd.read_csv('data/market1501_test.csv', names = ['pid', 'fid'], header = None, dtype = str)

In [0]:
num_images = len(labels)

height = 128
width = 64
net_input_size = (128,64)
channels = 3

### Reading the h5py Files

In [0]:
fileName = 'data_final.h5'
fileName_qt = 'data_qt.h5'

In [0]:
with h5py.File(fileName, "a") as out:
  X_train = np.asarray(out["X_train"])


In [0]:
with h5py.File(fileName_qt, "a") as out:
  X_query = np.asarray(out["X_dev"])
  X_test = np.asarray(out["X_test"])

In [0]:
print(X_train.shape)
print(X_query.shape)
print(X_test.shape)

(12936, 128, 64, 3)
(3368, 128, 64, 3)
(19732, 128, 64, 3)


## Set up core functions


### Train function
All models are trained with the same parameters and loss functions. Therefore a general function is made to perform the training for all models. 

In [0]:
def train(net, model_num):
  triplet_loss = TripletLoss(margin = None).cuda() # no margin means soft-margin

  ## optimizer
  logger.info('creating optimizer')
  optim = AdamOptimWrapper(net.parameters(), lr = 3e-4, wd = 0, t0 = 15000, t1 = 25000)

  ## dataloader
  selector = BatchHardTripletSelector()

  ds = Market1501(pids_list=list(labels.fid), array=X_train, is_train = True)
  sampler = BatchSampler(ds, 18, 4)
  dl = DataLoader(ds, batch_sampler = sampler, num_workers = 4)
  diter = iter(dl)

  logger.info('start training ...')
  loss_avg = []
  count = 0
  t_start = time.time()
  while True:
    try:
      imgs, lbs, _ = next(diter)
    except StopIteration:
      diter = iter(dl)
      imgs, lbs, _ = next(diter)

    net.train()
    imgs = imgs.cuda()
    lbs = lbs.cuda()
    embds = net(imgs)
    anchor, positives, negatives = selector(embds, lbs)

    loss = triplet_loss(anchor, positives, negatives)
    optim.zero_grad()
    loss.backward()
    optim.step()

    loss_avg.append(loss.detach().cpu().numpy())
    if count % 20 == 0 and count != 0:
      loss_avg = sum(loss_avg) / len(loss_avg)
      t_end = time.time()
      time_interval = t_end - t_start
      logger.info('iter: {}, loss: {:4f}, lr: {:4f}, time: {:3f}'.format(count, loss_avg, optim.lr, time_interval))
      loss_avg = []
      t_start = t_end

    count += 1
    if count == 25000: break

  ## dump model
  logger.info('saving trained model')
  torch.save(net.module.state_dict(), './res/model{}.pkl'.format(model_num))

  logger.info('everything finished')

## Create Embeddings
When the model is trained, it can be applied to both the Query and Test images. This is done by the function create_emb

In [0]:
def create_emb(dataset, fids, store_path, model_num):
  torch.multiprocessing.set_sharing_strategy('file_system')

  ## logging
  FORMAT = '%(levelname)s %(filename)s:%(lineno)d: %(message)s'
  logging.basicConfig(level=logging.INFO, format=FORMAT, stream=sys.stdout)
  logger = logging.getLogger(__name__)
  ## restore model
  logger.info('restoring model')
  model = net
  #model = nn.DataParallel(model)
  model = net.cuda()
  model.module.load_state_dict(torch.load('./res/model{}.pkl'.format(model_num)))
  model = nn.DataParallel(model)
  model.eval()

  ## load gallery dataset
  batchsize = 32
  ds = Market1501(pids_list=list(fids), array=dataset, is_train = False)
  dl = DataLoader(ds, batch_size = batchsize, drop_last = False, num_workers = 4)

  ## embedding samples
  logger.info('start embedding')
  all_iter_nums = len(ds) // batchsize + 1
  embeddings = []
  label_ids = []
  label_cams = []
  for it, (img, lb_id, lb_cam) in enumerate(dl):
    print('\r=======>  processing iter {} / {}'.format(it, all_iter_nums),
            end = '', flush = True)
    label_ids.append(lb_id)
    label_cams.append(lb_cam)
    embds = []
    for im in img:
        im = im.cuda()
        embd = model(im).detach().cpu().numpy()
        embds.append(embd)
    embed = sum(embds) / len(embds)
    embeddings.append(embed)
  print('  ...   completed')

  embeddings = np.vstack(embeddings)
  label_ids = np.hstack(label_ids)
  label_cams = np.hstack(label_cams)

  ## dump results
  logger.info('dump embeddings')
  embd_res = {'embeddings': embeddings, 'label_ids': label_ids, 'label_cams': label_cams}
  with open(store_path, 'wb') as fw:
    pickle.dump(embd_res, fw)

  logger.info('embedding finished')


In [0]:
from utils import pdist_np as pdist

In [0]:
def evaluate(test_embs, query_embs, cmc_rank):
    ## logging
    FORMAT = '%(levelname)s %(filename)s:%(lineno)d: %(message)s'
    logging.basicConfig(level=logging.INFO, format=FORMAT, stream=sys.stdout)
    logger = logging.getLogger(__name__)

    ## load embeddings
    logger.info('loading gallery embeddings')
    with open(test_embs, 'rb') as fr:
        gallery_dict = pickle.load(fr)
        emb_gallery, lb_ids_gallery, lb_cams_gallery = gallery_dict['embeddings'], gallery_dict['label_ids'], gallery_dict['label_cams']
    logger.info('loading query embeddings')
    with open(query_embs, 'rb') as fr:
        query_dict = pickle.load(fr)
        emb_query, lb_ids_query, lb_cams_query = query_dict['embeddings'], query_dict['label_ids'], query_dict['label_cams']

    ## compute and clean distance matrix
    dist_mtx = pdist(emb_query, emb_gallery)
    n_q, n_g = dist_mtx.shape
    indices = np.argsort(dist_mtx, axis = 1)
    matches = lb_ids_gallery[indices] == lb_ids_query[:, np.newaxis]
    matches = matches.astype(np.int32)
    all_aps = []
    all_cmcs = []
    logger.info('starting evaluating ...')
    for qidx in tqdm(range(n_q)):
        qpid = lb_ids_query[qidx]
        qcam = lb_cams_query[qidx]

        order = indices[qidx]
        pid_diff = lb_ids_gallery[order] != qpid
        cam_diff = lb_cams_gallery[order] != qcam
        useful = lb_ids_gallery[order] != -1
        keep = np.logical_or(pid_diff, cam_diff)
        keep = np.logical_and(keep, useful)
        match = matches[qidx][keep]

        if not np.any(match): continue

        cmc = match.cumsum()
        cmc[cmc > 1] = 1
        all_cmcs.append(cmc[:cmc_rank])

        num_real = match.sum()
        match_cum = match.cumsum()
        match_cum = [el / (1.0 + i) for i, el in enumerate(match_cum)]
        match_cum = np.array(match_cum) * match
        ap = match_cum.sum() / num_real
        all_aps.append(ap)

    assert len(all_aps) > 0, "NO QUERY MATCHED"
    mAP = sum(all_aps) / len(all_aps)
    all_cmcs = np.array(all_cmcs, dtype = np.float32)
    cmc = np.mean(all_cmcs, axis = 0)

    print('mAP is: {}, cmc is: {}'.format(mAP, cmc))

## Model log
Model 1 = baseline with stride = 2 in last layer 







model 6 = AA model with extra connected layers and stride = 2 in last layer


Model 7 = AA model filter = 2

Model 8 = AA Model filter = 5







## Baseline

In [0]:
torch.multiprocessing.set_sharing_strategy('file_system')
if not os.path.exists('./res'): os.makedirs('./res')

### Model 1
Baseline model with last conv layer using stride = 1.

In [0]:
from own_code.backbone_normal import EmbedNetwork

In [0]:
model_num = 1

In [0]:
logger.info('setting up backbone model and loss')
net = EmbedNetwork(pretrained_base=True).cuda()
net_1 = nn.DataParallel(net)
summary(net_1, (3,128,64))

setting up backbone model and loss
Downloading: "https://download.pytorch.org/models/resnet50-19c8e357.pth" to /root/.cache/torch/checkpoints/resnet50-19c8e357.pth
100%|██████████| 97.8M/97.8M [00:00<00:00, 179MB/s]


----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 64, 64, 32]           9,408
       BatchNorm2d-2           [-1, 64, 64, 32]             128
              ReLU-3           [-1, 64, 64, 32]               0
         MaxPool2d-4           [-1, 64, 32, 16]               0
            Conv2d-5           [-1, 64, 32, 16]           4,096
       BatchNorm2d-6           [-1, 64, 32, 16]             128
              ReLU-7           [-1, 64, 32, 16]               0
            Conv2d-8           [-1, 64, 32, 16]          36,864
       BatchNorm2d-9           [-1, 64, 32, 16]             128
             ReLU-10           [-1, 64, 32, 16]               0
           Conv2d-11          [-1, 256, 32, 16]          16,384
      BatchNorm2d-12          [-1, 256, 32, 16]             512
           Conv2d-13          [-1, 256, 32, 16]          16,384
      BatchNorm2d-14          [-1, 256,

In [0]:
train(net_1 = net, model_num = model_num)

In [0]:
create_emb(dataset = X_query, fids = labels_query.fid, model_num = 1, store_path= "./res/emb_query{}.pkl".format(model_num))
create_emb(dataset = X_test, fids = labels_test.fid, model_num = 1, store_path="./res/emb_test{}.pkl".format(model_num))

In [0]:
test_embs = "./res/emb_test{}.pkl".format(model_num)
query_embs = "./res/emb_query{}.pkl".format(model_num)
cmc_rank = 5
evaluate(test_embs = test_embs, query_embs = query_embs, cmc_rank = cmc_rank)

## anti-aliased models


### Model 6


In [0]:
torch.multiprocessing.set_sharing_strategy('file_system')
if not os.path.exists('./res'): os.makedirs('./res')

In [0]:
model_num = 6

In [0]:
filter_size = 3
net = models_lpf.resnet.resnet50(filter_size=filter_size)
net.load_state_dict(torch.load('models_lpf/resnet50_lpf%i.pth.tar'%filter_size)['state_dict'])
model = torch.nn.Sequential(*(list(net.children())[:-2]))

In [0]:
class DenseNormReLU(nn.Module):
    def __init__(self, in_feats, out_feats, *args, **kwargs):
        super(DenseNormReLU, self).__init__(*args, **kwargs)
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        self.dense = nn.Linear(in_features = in_feats, out_features = out_feats).to(self.device)
        self.bn = nn.BatchNorm1d(out_feats).to(self.device)
        self.relu = nn.ReLU(inplace = True).to(self.device)

    def forward(self, x):
        x = self.dense(x)
        x = self.bn(x)
        x = self.relu(x)
        return x
  
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
fc_head = DenseNormReLU(in_feats = 2048, out_feats = 1024)
embedding = nn.Linear(in_features = 1024, out_features = 128).to(device)

In [0]:
class Model(nn.Module):
  def __init__(self):
    super(Model, self).__init__()
    self.base = model
    self.fc_head = fc_head
    self.embedding = embedding

  def forward(self, x):
    # shape [N, C, H, W]
    x = self.base(x)
    x = F.avg_pool2d(x, x.size()[2:])
    x = x.contiguous().view(-1, 2048 )
    # shape [N, C]
    x = self.fc_head(x)
    x = self.embedding(x)

    return x

In [0]:
#device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = Model()
#model = Model().to(device)
model = model.cuda()
net = nn.DataParallel(model)
summary(net, (3, 128,64))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 64, 64, 32]           9,408
       BatchNorm2d-2           [-1, 64, 64, 32]             128
              ReLU-3           [-1, 64, 64, 32]               0
         MaxPool2d-4           [-1, 64, 63, 31]               0
   ReflectionPad2d-5           [-1, 64, 65, 33]               0
        Downsample-6           [-1, 64, 32, 16]               0
            Conv2d-7           [-1, 64, 32, 16]           4,096
       BatchNorm2d-8           [-1, 64, 32, 16]             128
              ReLU-9           [-1, 64, 32, 16]               0
           Conv2d-10           [-1, 64, 32, 16]          36,864
      BatchNorm2d-11           [-1, 64, 32, 16]             128
             ReLU-12           [-1, 64, 32, 16]               0
           Conv2d-13          [-1, 256, 32, 16]          16,384
      BatchNorm2d-14          [-1, 256,

In [0]:
train(net = net, model_num = model_num)

In [0]:
create_emb(dataset = X_query, fids = labels_query.fid, model_num = model_num, store_path= "./res/emb_query{}.pkl".format(model_num))
create_emb(dataset = X_test, fids = labels_test.fid, model_num = model_num, store_path="./res/emb_test{}.pkl".format(model_num))

=======>  processing iter 105 / 106  ...   completed
=======>  processing iter 616 / 617  ...   completed


In [0]:
test_embs = "./res/emb_test{}.pkl".format(model_num)
query_embs = "./res/emb_query{}.pkl".format(model_num)
cmc_rank = 5
evaluate(test_embs = test_embs, query_embs = query_embs, cmc_rank = cmc_rank)

100%|██████████| 3368/3368 [02:09<00:00, 26.08it/s]


mAP is: 0.7507753063839253, cmc is: [0.88182896 0.9162708  0.9334917  0.9435867  0.95190024]


### Model 7
AA model without GAP filter size = 2

In [0]:
torch.multiprocessing.set_sharing_strategy('file_system')
if not os.path.exists('./res'): os.makedirs('./res')

In [0]:
model_num = 7

In [0]:
filter_size = 2
net = models_lpf.resnet.resnet50(filter_size=filter_size)
net.load_state_dict(torch.load('models_lpf/resnet50_lpf%i.pth.tar'%filter_size)['state_dict'])
model = torch.nn.Sequential(*(list(net.children())[:-2]))

In [0]:
class DenseNormReLU(nn.Module):
    def __init__(self, in_feats, out_feats, *args, **kwargs):
        super(DenseNormReLU, self).__init__(*args, **kwargs)
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        self.dense = nn.Linear(in_features = in_feats, out_features = out_feats).to(self.device)
        self.bn = nn.BatchNorm1d(out_feats).to(self.device)
        self.relu = nn.ReLU(inplace = True).to(self.device)

    def forward(self, x):
        x = self.dense(x)
        x = self.bn(x)
        x = self.relu(x)
        return x
  
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
fc_head = DenseNormReLU(in_feats = 2048, out_feats = 1024)
embedding = nn.Linear(in_features = 1024, out_features = 128).to(device)

In [0]:
class Model(nn.Module):
  def __init__(self):
    super(Model, self).__init__()
    self.base = model
    self.fc_head = fc_head
    self.embedding = embedding

  def forward(self, x):
    # shape [N, C, H, W]
    x = self.base(x)
    x = F.avg_pool2d(x, x.size()[2:])
    x = x.contiguous().view(-1, 2048 )
    # shape [N, C]
    x = self.fc_head(x)
    x = self.embedding(x)

    return x

In [0]:
#device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = Model()
#model = Model().to(device)
model = model.cuda()
net = nn.DataParallel(model)
summary(net, (3, 128,64))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 64, 64, 32]           9,408
       BatchNorm2d-2           [-1, 64, 64, 32]             128
              ReLU-3           [-1, 64, 64, 32]               0
         MaxPool2d-4           [-1, 64, 63, 31]               0
   ReflectionPad2d-5           [-1, 64, 64, 32]               0
        Downsample-6           [-1, 64, 32, 16]               0
            Conv2d-7           [-1, 64, 32, 16]           4,096
       BatchNorm2d-8           [-1, 64, 32, 16]             128
              ReLU-9           [-1, 64, 32, 16]               0
           Conv2d-10           [-1, 64, 32, 16]          36,864
      BatchNorm2d-11           [-1, 64, 32, 16]             128
             ReLU-12           [-1, 64, 32, 16]               0
           Conv2d-13          [-1, 256, 32, 16]          16,384
      BatchNorm2d-14          [-1, 256,

In [0]:
train(net = net, model_num = model_num)

In [0]:
create_emb(dataset = X_query, fids = labels_query.fid, model_num = model_num, store_path= "./res/emb_query{}.pkl".format(model_num))
create_emb(dataset = X_test, fids = labels_test.fid, model_num = model_num, store_path="./res/emb_test{}.pkl".format(model_num))

In [0]:
test_embs = "./res/emb_test{}.pkl".format(model_num)
query_embs = "./res/emb_query{}.pkl".format(model_num)
cmc_rank = 5
evaluate(test_embs = test_embs, query_embs = query_embs, cmc_rank = cmc_rank)

### Model 8
AA model without GAP, filter size = 5

In [0]:
torch.multiprocessing.set_sharing_strategy('file_system')
if not os.path.exists('./res'): os.makedirs('./res')

In [0]:
model_num = 8

In [0]:
filter_size = 5
net = models_lpf.resnet.resnet50(filter_size=filter_size)
net.load_state_dict(torch.load('models_lpf/resnet50_lpf%i.pth.tar'%filter_size)['state_dict'])
model = torch.nn.Sequential(*(list(net.children())[:-2]))

In [0]:
class DenseNormReLU(nn.Module):
    def __init__(self, in_feats, out_feats, *args, **kwargs):
        super(DenseNormReLU, self).__init__(*args, **kwargs)
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        self.dense = nn.Linear(in_features = in_feats, out_features = out_feats).to(self.device)
        self.bn = nn.BatchNorm1d(out_feats).to(self.device)
        self.relu = nn.ReLU(inplace = True).to(self.device)

    def forward(self, x):
        x = self.dense(x)
        x = self.bn(x)
        x = self.relu(x)
        return x
  
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
fc_head = DenseNormReLU(in_feats = 2048, out_feats = 1024)
embedding = nn.Linear(in_features = 1024, out_features = 128).to(device)

In [0]:
class Model(nn.Module):
  def __init__(self):
    super(Model, self).__init__()
    self.base = model
    self.fc_head = fc_head
    self.embedding = embedding

  def forward(self, x):
    # shape [N, C, H, W]
    x = self.base(x)
    x = F.avg_pool2d(x, x.size()[2:])
    x = x.contiguous().view(-1, 2048 )
    # shape [N, C]
    x = self.fc_head(x)
    x = self.embedding(x)

    return x

In [0]:
#device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = Model()
#model = Model().to(device)
model = model.cuda()
net = nn.DataParallel(model)
summary(net, (3, 128,64))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 64, 64, 32]           9,408
       BatchNorm2d-2           [-1, 64, 64, 32]             128
              ReLU-3           [-1, 64, 64, 32]               0
         MaxPool2d-4           [-1, 64, 63, 31]               0
   ReflectionPad2d-5           [-1, 64, 67, 35]               0
        Downsample-6           [-1, 64, 32, 16]               0
            Conv2d-7           [-1, 64, 32, 16]           4,096
       BatchNorm2d-8           [-1, 64, 32, 16]             128
              ReLU-9           [-1, 64, 32, 16]               0
           Conv2d-10           [-1, 64, 32, 16]          36,864
      BatchNorm2d-11           [-1, 64, 32, 16]             128
             ReLU-12           [-1, 64, 32, 16]               0
           Conv2d-13          [-1, 256, 32, 16]          16,384
      BatchNorm2d-14          [-1, 256,

In [0]:
train(net = net, model_num = model_num)

In [0]:
create_emb(dataset = X_query, fids = labels_query.fid, model_num = model_num, store_path= "./res/emb_query{}.pkl".format(model_num))
create_emb(dataset = X_test, fids = labels_test.fid, model_num = model_num, store_path="./res/emb_test{}.pkl".format(model_num))

=======>  processing iter 105 / 106  ...   completed
=======>  processing iter 616 / 617  ...   completed


In [0]:
test_embs = "./res/emb_test{}.pkl".format(model_num)
query_embs = "./res/emb_query{}.pkl".format(model_num)
cmc_rank = 5
evaluate(test_embs = test_embs, query_embs = query_embs, cmc_rank = cmc_rank)

100%|██████████| 3368/3368 [02:43<00:00, 21.05it/s]


mAP is: 0.7417072430419447, cmc is: [0.8738124  0.91270787 0.9376485  0.9483373  0.9578385 ]


## Extra work

In [0]:
#checking GPU
print(torch.cuda.current_device())
print(torch.cuda.device(0))
print(torch.cuda.device_count())
print(torch.cuda.get_device_name(0))
print(torch.cuda.is_available())

0
1
Tesla P100-PCIE-16GB
True


Visualizing filters 

In [0]:
a = np.array([1., 1.])
filt_2 = a[:,None]*a[None,:]
b = np.array([1., 2., 1.])
filt_3 = b[:,None]*b[None,:]
c = np.array([1., 4., 6., 4., 1.])
filt_5 = c[:,None]*c[None,:]

print(filt_2, "\n",
      filt_3, "\n", filt_5)

[[1. 1.]
 [1. 1.]] 
 [[1. 2. 1.]
 [2. 4. 2.]
 [1. 2. 1.]] 
 [[ 1.  4.  6.  4.  1.]
 [ 4. 16. 24. 16.  4.]
 [ 6. 24. 36. 24.  6.]
 [ 4. 16. 24. 16.  4.]
 [ 1.  4.  6.  4.  1.]]


## Models not included in the paper

Model 2 = baseline with stride = 1 in last layer

Model 3 = Anti-aliased model, re-trained with stride = 2 in last layer

Model 4 = AA model with extra connected layers and stride = 2 in last layer. Model uses wrong way of cutting of layers.

Model 5 = AA model with extra connected layers, stride = 2 in last layer blurring before GAP. 

### Model 2 

Baseline model with last conv layer using stride = 1

In [0]:
from own_code.backbone import EmbedNetwork

In [0]:
model_num = 2

In [0]:
## model and loss
logger.info('setting up backbone model and loss')
net = EmbedNetwork(pretrained_base=True).cuda()
net = nn.DataParallel(net)


In [0]:
summary(net, (3,128,64))

In [0]:
train(net = net, model_num = 100)

In [0]:
embed(dataset = X_query, fids = labels_query.fid, store_path= "./res/emb_query{}.pkl".format(model_num))


In [0]:
embed(dataset = X_test, fids = labels_test.fid, store_path="./res/emb_test{}.pkl".format(model_num))

In [0]:
test_embs = "./res/emb_test{}.pkl".format(model_num)
query_embs = "./res/emb_query{}.pkl".format(model_num)
cmc_rank = 5
evaluate(test_embs = test_embs, query_embs = query_embs, cmc_rank = cmc_rank)

### Model 3
AA model with stride = 2 on last layer 

In [0]:
model_num = 3

In [0]:
class Model(nn.Module):
  def __init__(self):
    super(Model, self).__init__()
    self.base = model

  def forward(self, x):
    # shape [N, C, H, W]
    x = self.base(x)
    x = F.avg_pool2d(x, x.size()[2:])
    # shape [N, C]
    x = x.view(x.size(0), -1)

    return x

In [0]:
#device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = Model()
#model = Model().to(device)
model = model.cuda()
net = nn.DataParallel(model)

### Model 4
AA model with stride = 2 on last layer and extra layers added, similar to baseline

In [0]:
model_num = 4 

In [0]:
filter_size = 3
net = models_lpf.resnet.resnet50(filter_size=filter_size)
net.load_state_dict(torch.load('models_lpf/resnet50_lpf%i.pth.tar'%filter_size)['state_dict'])
model = torch.nn.Sequential(*(list(net.children())[:-1]))

In [0]:
class DenseNormReLU(nn.Module):
    def __init__(self, in_feats, out_feats, *args, **kwargs):
        super(DenseNormReLU, self).__init__(*args, **kwargs)
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        self.dense = nn.Linear(in_features = in_feats, out_features = out_feats).to(self.device)
        self.bn = nn.BatchNorm1d(out_feats).to(self.device)
        self.relu = nn.ReLU(inplace = True).to(self.device)

    def forward(self, x):
        x = self.dense(x)
        x = self.bn(x)
        x = self.relu(x)
        return x
  
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
fc_head = DenseNormReLU(in_feats = 2048, out_feats = 1024)
embed = nn.Linear(in_features = 1024, out_features = 128).to(device)


In [0]:
class Model(nn.Module):
  def __init__(self):
    super(Model, self).__init__()
    self.base = model
    self.fc_head = fc_head
    self.embed = embed

  def forward(self, x):
    # shape [N, C, H, W]
    x = self.base(x)
    x = F.avg_pool2d(x, x.size()[2:])
    x = x.contiguous().view(-1, 2048 )
    # shape [N, C]
    x = self.fc_head(x)
    x = self.embed(x)

    return x

In [0]:
#device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = Model()
#model = Model().to(device)
model = model.cuda()
net = nn.DataParallel(model)
summary(net, (3, 128,64))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 64, 64, 32]           9,408
       BatchNorm2d-2           [-1, 64, 64, 32]             128
              ReLU-3           [-1, 64, 64, 32]               0
         MaxPool2d-4           [-1, 64, 63, 31]               0
   ReflectionPad2d-5           [-1, 64, 65, 33]               0
        Downsample-6           [-1, 64, 32, 16]               0
            Conv2d-7           [-1, 64, 32, 16]           4,096
       BatchNorm2d-8           [-1, 64, 32, 16]             128
              ReLU-9           [-1, 64, 32, 16]               0
           Conv2d-10           [-1, 64, 32, 16]          36,864
      BatchNorm2d-11           [-1, 64, 32, 16]             128
             ReLU-12           [-1, 64, 32, 16]               0
           Conv2d-13          [-1, 256, 32, 16]          16,384
      BatchNorm2d-14          [-1, 256,

In [0]:
train(net = net, model_num = model_num)

In [0]:
create_embs(dataset = X_query, fids = labels_query.fid, store_path= "./res/emb_query{}.pkl".format(model_num))


In [0]:
embed(dataset = X_test, fids = labels_test.fid, store_path="./res/emb_test{}.pkl".format(model_num))

=======>  processing iter 616 / 617  ...   completed


In [0]:
test_embs = "./res/emb_test{}.pkl".format(model_num)
query_embs = "./res/emb_query{}.pkl".format(model_num)
cmc_rank = 5
evaluate(test_embs = test_embs, query_embs = query_embs, cmc_rank = cmc_rank)

100%|██████████| 3368/3368 [02:35<00:00, 21.72it/s]


mAP is: 0.7430370803791962, cmc is: [0.8770784  0.9141924  0.93171024 0.94447744 0.9513064 ]


### Model 5


In [0]:
torch.multiprocessing.set_sharing_strategy('file_system')
if not os.path.exists('./res'): os.makedirs('./res')

In [0]:
model_num = 5

In [0]:
filter_size = 3
net = models_lpf.resnet.resnet50(filter_size=filter_size)
net.load_state_dict(torch.load('models_lpf/resnet50_lpf%i.pth.tar'%filter_size)['state_dict'])
model = torch.nn.Sequential(*(list(net.children())[:-2]))

In [0]:
class DenseNormReLU(nn.Module):
    def __init__(self, in_feats, out_feats, *args, **kwargs):
        super(DenseNormReLU, self).__init__(*args, **kwargs)
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        self.dense = nn.Linear(in_features = in_feats, out_features = out_feats).to(self.device)
        self.bn = nn.BatchNorm1d(out_feats).to(self.device)
        self.relu = nn.ReLU(inplace = True).to(self.device)

    def forward(self, x):
        x = self.dense(x)
        x = self.bn(x)
        x = self.relu(x)
        return x
  
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
fc_head = DenseNormReLU(in_feats = 2048, out_feats = 1024)
embed = nn.Linear(in_features = 1024, out_features = 128).to(device)


In [0]:
blur = nn.Sequential(Downsample(filt_size = filter_size, channels = 2048, stride = 1),nn.ReLU(inplace=True), )

In [0]:
class Model(nn.Module):
  def __init__(self):
    super(Model, self).__init__()
    self.base = model
    self.blur = blur
    self.fc_head = fc_head
    self.embed = embed

  def forward(self, x):
    # shape [N, C, H, W]
    x = self.base(x)
    x = self.blur(x)
    x = F.avg_pool2d(x, x.size()[2:])
    x = x.contiguous().view(-1, 2048)
    # shape [N, C]
    x = self.fc_head(x)
    x = self.embed(x)

    return x

In [0]:
#device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = Model()
#model = Model().to(device)
model = model.cuda()
net = nn.DataParallel(model)
summary(net, (3, 128,64))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 64, 64, 32]           9,408
       BatchNorm2d-2           [-1, 64, 64, 32]             128
              ReLU-3           [-1, 64, 64, 32]               0
         MaxPool2d-4           [-1, 64, 63, 31]               0
   ReflectionPad2d-5           [-1, 64, 65, 33]               0
        Downsample-6           [-1, 64, 32, 16]               0
            Conv2d-7           [-1, 64, 32, 16]           4,096
       BatchNorm2d-8           [-1, 64, 32, 16]             128
              ReLU-9           [-1, 64, 32, 16]               0
           Conv2d-10           [-1, 64, 32, 16]          36,864
      BatchNorm2d-11           [-1, 64, 32, 16]             128
             ReLU-12           [-1, 64, 32, 16]               0
           Conv2d-13          [-1, 256, 32, 16]          16,384
      BatchNorm2d-14          [-1, 256,

In [0]:
train(net = net, model_num = model_num)

In [0]:
create_emb(dataset = X_query, fids = labels_query.fid, model_num = model_num, store_path= "./res/emb_query{}.pkl".format(model_num))
create_emb(dataset = X_test, fids = labels_test.fid, model_num = model_num, store_path="./res/emb_test{}.pkl".format(model_num))

=======>  processing iter 105 / 106  ...   completed
=======>  processing iter 616 / 617  ...   completed


In [0]:
test_embs = "./res/emb_test{}.pkl".format(model_num)
query_embs = "./res/emb_query{}.pkl".format(model_num)
cmc_rank = 5
evaluate(test_embs = test_embs, query_embs = query_embs, cmc_rank = cmc_rank)

100%|██████████| 3368/3368 [02:32<00:00, 22.04it/s]

mAP is: 0.747139206882985, cmc is: [0.87321854 0.91567695 0.9340855  0.94447744 0.952791  ]
